In [25]:
from glob import glob
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import requests
import seaborn as sns
from fredapi import Fred
from numba import float64, guvectorize, int64, njit, vectorize
from pandas.tseries.offsets import BMonthEnd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [26]:
#pd.set_option('display.max_columns', None)

In [27]:
def read_msci_data(filename, last_index):
    df = pd.read_excel(filename)
    df = df.iloc[6:last_index].copy()
    df = df.reset_index(drop=True)
    df.columns = ['date', 'price']
    df['date'] = pd.to_datetime(df['date'])
    df = df.replace(',','', regex=True)
    df['price'] = df['price'].astype(float)
    return df

In [28]:
df = read_msci_data('MSCI World.xls', 649)

In [29]:
df['drawdown'] = (df['price'] / df['price'].cummax() - 1)

In [30]:
def extract_financialtimes_data(filepaths):
    dfs = []
    for filepath in filepaths:
        dfs.append(pd.read_html(filepath)[2].iloc[::-1])
    df = pd.concat(dfs, ignore_index=True)
    df['Date'] = pd.to_datetime(df['Date'].apply(lambda x: ''.join(x.rsplit(',', maxsplit=2)[-2:])[1:]))
    df = df[df['Date'].isin(pd.date_range(df['Date'].iloc[0], df['Date'].iloc[-1], freq='BM'))]
    df = df.reset_index(drop=True)
    df = df[['Date', 'Close']]
    df.columns = ['date', 'price']
    return df
    

In [31]:
sti = extract_financialtimes_data(glob('STI Data/*/*.htm'))

In [32]:
fred = Fred()
fed_funds_rate = fred.get_series('DFF')
fed_funds_rate.name = 'ffr'

In [33]:
def custom_group_bounds(dt):
    end_date = dt + BMonthEnd(0)
    return end_date

In [34]:
fed_funds_rate = fed_funds_rate.div(36000).add(1).groupby(custom_group_bounds).prod().pow(12).sub(1).mul(100)

In [35]:
mas_api_url = 'https://eservices.mas.gov.sg/api/action/datastore/search.json'

In [36]:
usd_sgd_response = requests.get(mas_api_url,
                   params={'resource_id': '10eafb90-11a2-4fbd-b7a7-ac15a42d60b6',
                           'between[end_of_month]': f'1969-12,{pd.to_datetime("today").strftime("%Y-%m")}',
                           'fields': 'end_of_month,usd_sgd'
                           }
                   ).json()

In [37]:
usdsgd = pd.DataFrame(usd_sgd_response['result']['records'])[['end_of_month', 'usd_sgd']]

In [38]:
usdsgd['end_of_month'] = pd.to_datetime(usdsgd['end_of_month']) + BMonthEnd()

In [39]:
def download_sgd_interest_rates():
    offset = 0
    dfs = []
    while True:
        sgd_interest_rates_response = requests.get(mas_api_url,
                    params={'resource_id': '9a0bf149-308c-4bd2-832d-76c8e6cb47ed',
                            'between[end_of_day]': f'1987-07-01,{pd.to_datetime("today").strftime("%Y-%m-%d")}',
                            'offset': f'{offset}',
                            'fields': 'end_of_day,interbank_overnight,sora'
                            }
                    ).json()
        df = pd.DataFrame(sgd_interest_rates_response['result']['records'])[['end_of_day', 'interbank_overnight', 'sora']]
        offset += 100
        dfs.append(df)
        if len(df) < 100:
            break
    sgd_interest_rates = pd.concat(dfs)
    sgd_interest_rates['interbank_overnight'] = sgd_interest_rates['interbank_overnight'].astype(float)
    sgd_interest_rates['end_of_day'] = pd.to_datetime(sgd_interest_rates['end_of_day'])
    sgd_interest_rates = sgd_interest_rates.dropna(how='all', subset=['interbank_overnight', 'sora'])
    sgd_interest_rates = sgd_interest_rates.drop_duplicates().drop_duplicates(subset=['end_of_day', 'interbank_overnight']).drop_duplicates(subset=['end_of_day', 'sora'])
    sgd_interest_rates = sgd_interest_rates.reset_index(drop=True)
    return sgd_interest_rates

In [40]:
def load_sgd_interest_rates():
    try:
        sgd_interest_rates = pd.read_csv('sgd_interest_rates.csv', parse_dates=['end_of_day'])
        if pd.to_datetime(sgd_interest_rates['end_of_day']).iloc[-1] < pd.to_datetime('today') + BMonthEnd(-1):
            raise FileNotFoundError
        return sgd_interest_rates
    except FileNotFoundError:
        sgd_interest_rates = download_sgd_interest_rates()
        sgd_interest_rates.to_csv('sgd_interest_rates.csv', index=False)
        return sgd_interest_rates

In [41]:
sgd_interest_rates = load_sgd_interest_rates()

In [42]:
sgd_interest_rates = sgd_interest_rates.set_index('end_of_day').resample('D').ffill().div(36000).add(1).groupby(custom_group_bounds).prod().pow(12).sub(1).mul(100).replace(0, np.nan)

In [43]:
sgd_interest_rates.loc['2014-01-31', 'interbank_overnight'] = np.nan

In [44]:
sgd_interest_rates['sgd_ir_1m'] = sgd_interest_rates['interbank_overnight'].fillna(sgd_interest_rates['sora'])

In [45]:
sgd_interest_rates

,interbank_overnight,sora,sgd_ir_1m
end_of_day,,,
1987-07-31,3.432429,NaN,3.432429
1987-08-31,2.380101,NaN,2.380101
1987-09-30,2.339506,NaN,2.339506
1987-10-30,3.015769,NaN,3.015769
1987-11-30,3.381409,NaN,3.381409
...,...,...,...
2023-04-28,NaN,3.331171,3.331171
2023-05-31,NaN,4.063246,4.063246
2023-06-30,NaN,3.820647,3.820647


In [46]:
df = df.merge(fed_funds_rate, left_on='date', right_index=True)

In [47]:
df = df.merge(sgd_interest_rates['sgd_ir_1m'], how='left',left_on='date', right_index=True)

In [84]:
periods = ['1m', '3m', '6m', '1y', '2y', '3y', '5y', '10y', '15y', '20y', '25y', '30y']
durations = [1, 3, 6, 12, 24, 36, 60, 120, 180, 240, 300, 360]

In [85]:
@njit
def calculate_return(ending_index, dca_length, monthly_returns, investment_horizon=None):
    if investment_horizon is None:
        investment_horizon = dca_length
    elif investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    if ending_index < dca_length:
        return np.nan
    share_value = 0
    cash = 1
    for i in range(ending_index - investment_horizon, ending_index - investment_horizon + dca_length):
        cash -= 1/dca_length
        share_value += 1/dca_length
        share_value *= 1 + monthly_returns[i+1]
    for i in range(ending_index - investment_horizon + dca_length, ending_index):
        share_value *= 1 + monthly_returns[i+1]
    return share_value - 1

@guvectorize([(int64, float64[:], int64, float64[:])], '(),(n),()->(n)', target='parallel', nopython=True)
def calculate_return_vector(dca_length, monthly_returns, investment_horizon, res=np.array([])):
    if investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    for i in range(len(monthly_returns)):
        if i < investment_horizon:
            res[i] = np.nan
        share_value = 0
        cash = 1
        for j in range(i - investment_horizon, i - investment_horizon + dca_length):
            cash -= 1/dca_length
            share_value += 1/dca_length
            share_value *= 1 + monthly_returns[j+1]
        for j in range(i - investment_horizon + dca_length, i):
            share_value *= 1 + monthly_returns[j+1]
        res[i] = share_value - 1
        
@guvectorize([(float64, float64, float64, float64, int64, int64, int64, float64[:], float64[:])], '(),(),(),(),(),(),(),(n)->(n)', target='parallel', nopython=True)
def calculate_lumpsum_return_with_fees_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, total_investment, dca_length, dca_interval, investment_horizon, monthly_returns, res=np.array([])):
    if investment_horizon < dca_length:
        raise ValueError('Investment horizon must be greater than or equal to DCA length')
    if fixed_transaction_fees >= total_investment / dca_length * dca_interval:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < investment_horizon:
            res[i] = np.nan
        share_value = 0
        cash = total_investment
        dca_amount = total_investment / dca_length * dca_interval
        for j in range(i - investment_horizon, i - investment_horizon + dca_length, dca_interval):
            cash -= dca_amount
            share_value += dca_amount * (1 - variable_transaction_fees) - fixed_transaction_fees
            for k in range(j, j + dca_interval):
                share_value *= ((1 + monthly_returns[k+1]) ** 12 - annualised_holding_fees) ** (1/12)
        if cash != 0.:
            share_value += cash * (1 - variable_transaction_fees) - fixed_transaction_fees
            cash-= cash
        for j in range(i - investment_horizon + dca_length, i):
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
        res[i] = (share_value - total_investment) / total_investment
        
@guvectorize([(float64, float64, float64, float64, int64, int64, float64[:], float64[:])], '(),(),(),(),(),(),(n)->(n)', target='parallel', nopython=True)
def calculate_dca_return_with_fees_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, monthly_amount, dca_length, dca_interval, monthly_returns, res=np.array([])):
    total_investment = monthly_amount * dca_length
    dca_amount = monthly_amount * dca_interval
    if fixed_transaction_fees >= dca_amount:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < dca_length:
            res[i] = np.nan
        share_value = 0
        amount_invested = 0
        for index, j in enumerate(range(i - dca_length, i)):
            if (index + 1) % dca_interval == 0:
                share_value += dca_amount * (1 - variable_transaction_fees) - fixed_transaction_fees
                amount_invested += dca_amount
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
        res[i] = (share_value - amount_invested) / total_investment

@guvectorize([(float64, float64, float64, float64, float64, int64, float64[:], float64[:], float64[:], float64[:])], '(),(),(),(),(),(),(n),(n),(n)->(n)', target='parallel', nopython=True)
def calculate_dca_buythedip_return_with_fees_vector(variable_transaction_fees, fixed_transaction_fees, annualised_holding_fees, monthly_investment, monthly_savings, dca_length, monthly_returns, fed_funds_rate, drawdown, res=np.array([])):
    total_investment = (monthly_investment + monthly_savings) * dca_length
    if fixed_transaction_fees >= monthly_investment:
        raise ValueError('Fixed fees must be less than the amount invested in each DCA')
    for i in range(len(monthly_returns)):
        if i < dca_length:
            res[i] = np.nan
        share_value = 0
        amount_invested = 0
        warchest = 0
        for j in range(i - dca_length, i):
            share_value += monthly_investment * (1 - variable_transaction_fees) - fixed_transaction_fees
            amount_invested += monthly_investment
            warchest += monthly_savings
            if drawdown[j] < -0.20:
                share_value += warchest * (1 - variable_transaction_fees) - fixed_transaction_fees
                amount_invested += warchest
                warchest = 0
            warchest *= (1 + fed_funds_rate[j+1] / 100) ** (1/12)
            share_value *= ((1 + monthly_returns[j+1]) ** 12 - annualised_holding_fees) ** (1/12)
        res[i] = (share_value - amount_invested) / total_investment

In [86]:
def add_return_columns(df):
    for period, duration in zip(periods, durations):
        df[f'{period}_cumulative'] = df['price'].pct_change(periods=duration)
    for period, duration in zip(periods, durations):
        df[f'{period}_annualized'] = (1 + df[f'{period}_cumulative'])**(12/duration) - 1
    for period, duration in zip(periods, durations):
        df[f'{period}_dca_cumulative'] = calculate_return_vector(duration, df['1m_cumulative'].values, duration)
    for period, duration in zip(periods, durations):
        df[f'{period}_dca_annualized'] = (1 + df[f'{period}_dca_cumulative'])**(12/duration) - 1
    for period, duration in zip(periods, durations):
        df[f'{period}_cumulative_difference'] = df[f'{period}_cumulative'] - df[f'{period}_dca_cumulative']
    for period, duration in zip(periods, durations):
        df[f'{period}_difference_in_annualized'] = df[f'{period}_annualized'] - df[f'{period}_dca_annualized']

In [87]:
add_return_columns(df)

In [88]:
add_return_columns(sti)

In [89]:
df.head(10)

,date,price,drawdown,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,...,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,15y_difference_in_annualized,20y_difference_in_annualized,25y_difference_in_annualized,30y_difference_in_annualized
211,1987-07-31,942.907,0.000000,7.037649,3.432429,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,1987-08-31,998.805,0.000000,7.203655,2.380101,0.059283,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,1987-09-30,981.574,-0.017252,7.482310,2.339506,-0.017252,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,1987-10-30,815.069,-0.183956,7.584087,3.015769,-0.169631,-0.135579,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,1987-11-30,795.400,-0.203648,7.157933,3.381409,-0.024132,-0.203648,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,1987-12-31,830.015,-0.168992,7.245107,2.718167,0.043519,-0.154404,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,1988-01-29,850.427,-0.148556,6.831615,2.712017,0.024592,0.043380,-0.098080,NaN,NaN,...,-0.107151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,1988-02-29,899.942,-0.098981,7.052989,2.964982,0.058224,0.131433,-0.098981,NaN,NaN,...,-0.254527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
219,1988-03-31,927.284,-0.071607,7.030515,2.998622,0.030382,0.117189,-0.055309,NaN,NaN,...,-0.276164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,1988-04-29,939.151,-0.059725,6.863664,2.724675,0.012798,0.104329,0.152235,NaN,NaN,...,0.108408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df.describe()

,price,drawdown,ffr,sgd_ir_1m,1m_cumulative,3m_cumulative,6m_cumulative,1y_cumulative,2y_cumulative,3y_cumulative,...,6m_difference_in_annualized,1y_difference_in_annualized,2y_difference_in_annualized,3y_difference_in_annualized,5y_difference_in_annualized,10y_difference_in_annualized,15y_difference_in_annualized,20y_difference_in_annualized,25y_difference_in_annualized,30y_difference_in_annualized
count,432.000000,432.000000,432.000000,432.000000,431.000000,429.000000,426.000000,420.000000,408.000000,396.000000,...,426.000000,420.000000,408.000000,396.000000,372.000000,312.000000,252.000000,192.000000,132.000000,72.000000
mean,4396.697900,-0.093627,3.241275,1.660941,0.007189,0.021304,0.044354,0.090427,0.190155,0.291280,...,0.044977,0.041801,0.040729,0.038792,0.036307,0.034422,0.030065,0.030896,0.033132,0.033910
std,3275.951244,0.117523,2.824053,1.668261,0.044290,0.076614,0.109320,0.160906,0.231263,0.286215,...,0.119263,0.090879,0.065577,0.052649,0.037789,0.020579,0.013755,0.010483,0.004923,0.003085
min,795.400000,-0.536504,0.048345,0.017001,-0.189341,-0.331171,-0.433765,-0.467637,-0.467822,-0.450059,...,-0.326142,-0.340453,-0.168074,-0.115756,-0.061955,-0.016487,-0.003166,0.004933,0.019341,0.027770
25%,1921.943750,-0.138815,0.241541,0.254448,-0.017722,-0.012016,-0.015845,-0.000692,0.064296,0.137873,...,-0.022948,-0.004865,0.008914,0.016442,0.018465,0.021906,0.020150,0.024764,0.030361,0.031282
50%,3452.812500,-0.043873,2.743870,1.116429,0.012842,0.029573,0.053305,0.123653,0.222210,0.322392,...,0.049325,0.050570,0.054301,0.050286,0.043054,0.038801,0.031897,0.033661,0.033922,0.034409
75%,6149.143500,0.000000,5.563702,2.729043,0.032758,0.067083,0.110132,0.188915,0.360828,0.497271,...,0.111158,0.094242,0.082459,0.075922,0.064007,0.050024,0.041268,0.038672,0.036434,0.036207
max,14223.137000,0.000000,11.392739,8.139264,0.128278,0.307832,0.472122,0.551807,0.897989,0.904105,...,0.659631,0.398270,0.240770,0.170829,0.125724,0.085297,0.053585,0.047239,0.042055,0.039565


In [91]:
df.loc[:, [*df.loc[:,'1m_annualized':'30y_annualized'].columns, *df.loc[:,'1m_dca_annualized':'30y_dca_annualized']]].describe()

,1m_annualized,3m_annualized,6m_annualized,1y_annualized,2y_annualized,3y_annualized,5y_annualized,10y_annualized,15y_annualized,20y_annualized,...,6m_dca_annualized,1y_dca_annualized,2y_dca_annualized,3y_dca_annualized,5y_dca_annualized,10y_dca_annualized,15y_dca_annualized,20y_dca_annualized,25y_dca_annualized,30y_dca_annualized
count,431.000000,429.000000,426.000000,420.000000,408.000000,396.000000,372.000000,312.000000,252.000000,192.000000,...,426.000000,420.000000,408.000000,396.000000,372.000000,312.000000,252.000000,192.000000,132.000000,72.000000
mean,0.223382,0.123581,0.102598,0.090427,0.085253,0.082273,0.080024,0.076253,0.067414,0.068795,...,0.057621,0.048626,0.044525,0.043482,0.043717,0.041830,0.037349,0.037899,0.040902,0.044769
std,0.605130,0.317959,0.222108,0.160906,0.111406,0.086347,0.060808,0.032816,0.017443,0.011913,...,0.138053,0.094993,0.065836,0.052007,0.037958,0.021570,0.013169,0.009337,0.004068,0.003316
min,-0.919451,-0.799894,-0.679378,-0.467637,-0.270495,-0.180708,-0.053404,-0.021078,0.032600,0.037583,...,-0.511205,-0.354103,-0.243771,-0.168423,-0.085712,-0.028866,-0.006182,0.009435,0.032273,0.036479
25%,-0.193113,-0.047206,-0.031439,-0.000692,0.031647,0.043994,0.031038,0.057263,0.052422,0.060047,...,-0.011918,0.000548,0.014287,0.018719,0.025612,0.029056,0.027822,0.031995,0.038197,0.042745
50%,0.165462,0.123645,0.109452,0.123653,0.105536,0.097623,0.082909,0.077134,0.067863,0.069758,...,0.070507,0.063890,0.062920,0.055939,0.048690,0.042335,0.039587,0.037769,0.040847,0.044024
75%,0.472260,0.296562,0.232393,0.188915,0.166545,0.144019,0.125314,0.101212,0.078781,0.075706,...,0.142814,0.110151,0.090579,0.079621,0.070275,0.054557,0.045911,0.043814,0.042392,0.046162
max,3.255917,1.925556,1.167144,0.551807,0.377675,0.239454,0.206518,0.142141,0.107792,0.096713,...,0.507513,0.248874,0.154601,0.124698,0.113149,0.092094,0.064799,0.058883,0.051644,0.052056


In [92]:
go.Figure(
    data = [
        go.Box(
            x=df[column],
            name=column,
            )
        for column in df.loc[:,'1m_annualized':'30y_annualized'].columns
    ],
    layout = go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [93]:
go.Figure(
    data = [
        go.Box(
            x=df[column],
            name=column,
            )
        for column in chain.from_iterable(zip(df.loc[:,'1m_annualized':'30y_annualized'].columns, df.loc[:,'1m_dca_annualized':'30y_dca_annualized']))
    ],
    layout = go.Layout(
        height=800,
        xaxis=dict(
            tickformat='.2%',
        )
    )
)

In [94]:
go.Figure(
    [
        go.Scatter(
            x=df['date'],
            y=df[column],
            name=column,
            mode='lines'
            )
        for column in ['5y_annualized', '5y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [95]:
go.Figure(
    [
        go.Scatter(
            x=df['date'],
            y=df[column],
            name=column,
            mode='lines'
            )
        for column in ['10y_annualized', '10y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [96]:
go.Figure(
    [
        go.Scatter(
            x=df['date'],
            y=df[column],
            name=column,
            mode='lines'
            )
        for column in ['20y_annualized', '20y_dca_annualized']
    ],
    layout = go.Layout(
        yaxis=dict(
            tickformat='.0%',
        )
    )
)

In [97]:
go.Figure(
    [
        go.Box(
            x=df[column],
            name=column,
            opacity=0.75
            )
        for column in df.loc[:, '1m_difference_in_annualized':'30y_difference_in_annualized'].columns
    ],
    layout = go.Layout(
        xaxis=dict(
            tickformat='.0%',
        )
    )
)